In [45]:
import logging
import json
import os
import tempfile
import azure.functions as func
import azure.durable_functions as df
from moviepy.editor import *
import itertools
from azure.storage.blob import BlobServiceClient

tempdir = tempfile.gettempdir()
def dowload_file(name, tempdir = tempdir):

    blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=aistudio9f70;AccountKey=1EtoBzcsmU6AfEQTeAKwi5oiKWVkAUU4y96DkExaY6dXZOjsWuJYtFI/19zk0ipL/iF9aD65W6JM+ASt5GayTg==;EndpointSuffix=core.windows.net")
    data_container_client = blob_service_client.get_container_client("videos")
    tosave = os.path.join(tempdir, name)
    with open(tosave, "wb") as data:
        download_stream = data_container_client.download_blob(name)
        data.write(download_stream.readall())
    return name

In [46]:
import math
from pydub import AudioSegment
import os
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '\\' + filename
        self.output_folder = folder + '\\' + 'splitted'
        self.audio = AudioSegment.from_wav(self.filepath)

    def get_duration(self):
        return self.audio.duration_seconds

    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        filepath = self.output_folder + '\\' + split_filename

        split_audio.export(filepath, format="wav")
        return filepath

    def multiple_split(self, min_per_split):
        files = []
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            files.append(self.single_split(i, i+min_per_split, split_fn))
            print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                print('All splited successfully')
        return files

In [47]:
fileUrl = "videoplayback.mp4"
audiopath = dowload_file(fileUrl)
video = VideoFileClip(os.path.join(tempdir, fileUrl)) # 2.
audio = video.audio # 3.
audiourl = fileUrl.replace(".mp4", ".wav")
audio.write_audiofile(os.path.join(tempdir, audiourl))
split_wav = SplitWavAudioMubin(tempdir, audiourl)
os.mkdir(tempdir + '\\' + 'splitted')
results_paths = split_wav.multiple_split(min_per_split=2)

MoviePy - Writing audio in C:\Users\CESARD~1.SOT\AppData\Local\Temp\videoplayback.wav


MoviePy - Done.
0 Done
2 Done
4 Done
6 Done


In [48]:
results_paths

['C:\\Users\\CESARD~1.SOT\\AppData\\Local\\Temp\\splitted\\0_videoplayback.wav',
 'C:\\Users\\CESARD~1.SOT\\AppData\\Local\\Temp\\splitted\\2_videoplayback.wav',
 'C:\\Users\\CESARD~1.SOT\\AppData\\Local\\Temp\\splitted\\4_videoplayback.wav',
 'C:\\Users\\CESARD~1.SOT\\AppData\\Local\\Temp\\splitted\\6_videoplayback.wav']

In [44]:
os.path.join("tmpdir", fileUrl)

'tmpdir\\videoplayback.mp4'

In [37]:
split_wav = SplitWavAudioMubin("tmpdir", audiourl)

In [27]:
from moviepy.editor import *
video = VideoFileClip("tmpdir/videoplayback.mp4") # 2.
audio = video.audio # 3.
audio.write_audiofile("tmpdir/videoplayback.mp3") 

MoviePy - Writing audio in tmpdir/videoplayback.mp3


MoviePy - Done.


In [24]:
audio = AudioSegment.from_mp3(r"C:\Users\cesard.soto\Desktop\VideoTrans\tmpdir\videoplayback.mp4")


FileNotFoundError: [WinError 2] The system cannot find the file specified